## Stage 3: Predictive Data Analytics

### Imports and Spark session

In [1]:
from pyspark.sql import SparkSession

team = "team15"
warehouse = "project/hive/warehouse"

spark = SparkSession.builder\
        .appName(f"{team} - spark ML")\
        .master("yarn")\
        .config("hive.metastore.uris", "thrift://hadoop-02.uni.innopolis.ru:9883")\
        .config("spark.sql.warehouse.dir", warehouse)\
        .enableHiveSupport()\
        .getOrCreate()

print("Spark Session Created.")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/08 16:51:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/08 16:51:45 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
25/05/08 16:51:45 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


Spark Session Created.


### Load data from Hive

In [2]:
# List all databases

spark.sql("SHOW DATABASES;").show()

+--------------------+
|           namespace|
+--------------------+
|             default|
|             retake1|
|             root_db|
|                show|
|     team0_projectdb|
|    team11_projectdb|
|           team12_db|
|team12_hive_proje...|
|    team12_projectdb|
|    team13_projectdb|
|    team14_projectdb|
|    team15_projectdb|
|    team16_projectdb|
|    team18_projectdb|
|    team19_projectdb|
|     team1_projectdb|
|    team20_projectdb|
| team21_projectdb_v2|
| team21_projectdb_v3|
| team21_projectdb_v4|
+--------------------+
only showing top 20 rows



In [3]:
# List all tables

hive_db_name = f"{team}_projectdb"
spark.sql(f"USE {hive_db_name};")
spark.sql("SHOW TABLES;").show()

+----------------+--------------------+-----------+
|       namespace|           tableName|isTemporary|
+----------------+--------------------+-----------+
|team15_projectdb|             airport|      false|
|team15_projectdb|   airport_optimized|      false|
|team15_projectdb|  cancellationreason|      false|
|team15_projectdb|cancellationreaso...|      false|
|team15_projectdb|              flight|      false|
|team15_projectdb|    flight_optimized|      false|
|team15_projectdb|          q1_results|      false|
|team15_projectdb|          q2_results|      false|
|team15_projectdb|          q3_results|      false|
|team15_projectdb|          q4_results|      false|
|team15_projectdb|          q5_results|      false|
|team15_projectdb|          q6_results|      false|
+----------------+--------------------+-----------+



In [4]:
# Read Hive tables

airport = spark.read.format("avro").table(f'{hive_db_name}.airport')
flight = spark.read.format("avro").table(f'{hive_db_name}.flight')
cancellationreason = spark.read.format("avro").table(f'{hive_db_name}.cancellationreason')

In [5]:
# Run some queries

airport.printSchema()
flight.printSchema()
cancellationreason.printSchema()

root
 |-- code: string (nullable = true)

root
 |-- flightid: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- dayofmonth: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- deptime: double (nullable = true)
 |-- crsdeptime: double (nullable = true)
 |-- arrtime: double (nullable = true)
 |-- crsarrtime: double (nullable = true)
 |-- actualelapsedtime: double (nullable = true)
 |-- crselapsedtime: double (nullable = true)
 |-- airtime: double (nullable = true)
 |-- arrdelay: double (nullable = true)
 |-- depdelay: double (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- distance: double (nullable = true)
 |-- taxiin: double (nullable = true)
 |-- taxiout: double (nullable = true)
 |-- cancelled: double (nullable = true)
 |-- cancellationcode: string (nullable = true)
 |-- diverted: double (nullable = true)

root
 |-- code: string (nullable = true)
 |-- descripti

In [6]:
spark.sql("SELECT flightid, cancelled, cancellationcode FROM flight LIMIT 10").show()

25/05/08 16:52:06 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


+--------+---------+----------------+
|flightid|cancelled|cancellationcode|
+--------+---------+----------------+
| 6617976|      0.0|            NULL|
| 6617977|      0.0|            NULL|
| 6617978|      0.0|            NULL|
| 6617979|      0.0|            NULL|
| 6617980|      0.0|            NULL|
| 6617981|      0.0|            NULL|
| 6617982|      0.0|            NULL|
| 6617983|      0.0|            NULL|
| 6617984|      0.0|            NULL|
| 6617985|      0.0|            NULL|
+--------+---------+----------------+



In [7]:
spark.sql("SELECT * FROM airport LIMIT 10").show()

+----+
|code|
+----+
| ABE|
| ABI|
| ABQ|
| ABR|
| ABY|
| ACK|
| ACT|
| ACV|
| ACY|
| ADK|
+----+



In [8]:
spark.sql("SELECT * FROM cancellationreason LIMIT 10").show()

+----+-----------+
|code|description|
+----+-----------+
|   A|    Carrier|
|   B|    Weather|
|   C|        NAS|
|   D|   Security|
+----+-----------+



### Data prep for ML modeling

In [9]:
import math
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, sin, cos, pi, hour, minute, dayofmonth, dayofweek, year as f_year
from pyspark.sql.types import IntegerType, StringType, DoubleType
from pyspark.sql import functions as F

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.mllib.evaluation import MulticlassMetrics

In [10]:
hive_table_name = "flight"
df = spark.table(f"{hive_db_name}.{hive_table_name}")
df.printSchema()

root
 |-- flightid: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- dayofmonth: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- deptime: double (nullable = true)
 |-- crsdeptime: double (nullable = true)
 |-- arrtime: double (nullable = true)
 |-- crsarrtime: double (nullable = true)
 |-- actualelapsedtime: double (nullable = true)
 |-- crselapsedtime: double (nullable = true)
 |-- airtime: double (nullable = true)
 |-- arrdelay: double (nullable = true)
 |-- depdelay: double (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- distance: double (nullable = true)
 |-- taxiin: double (nullable = true)
 |-- taxiout: double (nullable = true)
 |-- cancelled: double (nullable = true)
 |-- cancellationcode: string (nullable = true)
 |-- diverted: double (nullable = true)



In [11]:
df.show(10)

+--------+----+-----+----------+---------+-------+----------+-------+----------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+
|flightid|year|month|dayofmonth|dayofweek|deptime|crsdeptime|arrtime|crsarrtime|actualelapsedtime|crselapsedtime|airtime|arrdelay|depdelay|origin|dest|distance|taxiin|taxiout|cancelled|cancellationcode|diverted|
+--------+----+-----+----------+---------+-------+----------+-------+----------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+
| 6617976|2017|   11|        12|        7| 2049.0|    2055.0| 2144.0|    2205.0|            115.0|         130.0|   97.0|   -21.0|    -6.0|   DCA| MKE|   634.0|   3.0|   15.0|      0.0|            NULL|     0.0|
| 6617977|2017|   11|        12|        7| 1430.0|    1430.0| 1523.0|    1535.0|            113.0|         125.0|  100.0|   -12.0|     0.0|   DCA| MKE| 

In [12]:
print(f"Total number of rows: {df.count()}")

Total number of rows: 18505725


#### Selecting features (drop unnecessary columns)

In [13]:
columns = df.columns
print(columns)

['flightid', 'year', 'month', 'dayofmonth', 'dayofweek', 'deptime', 'crsdeptime', 'arrtime', 'crsarrtime', 'actualelapsedtime', 'crselapsedtime', 'airtime', 'arrdelay', 'depdelay', 'origin', 'dest', 'distance', 'taxiin', 'taxiout', 'cancelled', 'cancellationcode', 'diverted']


In [14]:
null_counts = df.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in df.columns
])

null_counts.show()

+--------+----+-----+----------+---------+-------+----------+-------+----------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+
|flightid|year|month|dayofmonth|dayofweek|deptime|crsdeptime|arrtime|crsarrtime|actualelapsedtime|crselapsedtime|airtime|arrdelay|depdelay|origin|dest|distance|taxiin|taxiout|cancelled|cancellationcode|diverted|
+--------+----+-----+----------+---------+-------+----------+-------+----------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+
|       0|   0|    0|         0|        0| 256081|         0| 271763|         0|           309166|            23| 309166|  311764|  261033|     0|   0|       0|271764| 263393|        0|        18240587|       0|
+--------+----+-----+----------+---------+-------+----------+-------+----------+-----------------+--------------+-------+--------+--------+------+----+-

In [15]:
features = [
    'year', 'month', 'dayofmonth', 'dayofweek',      # timestamps (will be transformed later)
    'crsdeptime', 'crsarrtime',                      # timestamps (will be transformed later)
    'crselapsedtime',
    'origin', 'dest', 'distance',
]
label = "cancelled"

df = df.select(features + [label])
df = df.withColumn(label, F.col(label).cast(DoubleType()))
df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- dayofmonth: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- crsdeptime: double (nullable = true)
 |-- crsarrtime: double (nullable = true)
 |-- crselapsedtime: double (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- distance: double (nullable = true)
 |-- cancelled: double (nullable = true)



In [16]:
df.show(10)

+----+-----+----------+---------+----------+----------+--------------+------+----+--------+---------+
|year|month|dayofmonth|dayofweek|crsdeptime|crsarrtime|crselapsedtime|origin|dest|distance|cancelled|
+----+-----+----------+---------+----------+----------+--------------+------+----+--------+---------+
|2017|   11|        12|        7|    2055.0|    2205.0|         130.0|   DCA| MKE|   634.0|      0.0|
|2017|   11|        12|        7|    1430.0|    1535.0|         125.0|   DCA| MKE|   634.0|      0.0|
|2017|   11|        12|        7|    1305.0|    1500.0|         175.0|   DCA| MSY|   969.0|      0.0|
|2017|   11|        12|        7|    1855.0|    2050.0|         175.0|   DCA| MSY|   969.0|      0.0|
|2017|   11|        12|        7|    1645.0|    1845.0|         180.0|   DCA| OMA|  1012.0|      0.0|
|2017|   11|        12|        7|    1955.0|    2115.0|          80.0|   DCA| PVD|   356.0|      0.0|
|2017|   11|        12|        7|    1145.0|    1305.0|          80.0|   DCA| PVD|

In [17]:
from pyspark.sql.functions import sum

df.select(sum("cancelled")).show()

+--------------+
|sum(cancelled)|
+--------------+
|      265138.0|
+--------------+



#### Dataset preprocessing

1. Handle missing values

In [18]:
df = df.na.drop()
print(f"Number of rows after NA drop: {df.count()}")

Number of rows after NA drop: 18505702


In [19]:
null_counts = df.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in df.columns
])

null_counts.show()

+----+-----+----------+---------+----------+----------+--------------+------+----+--------+---------+
|year|month|dayofmonth|dayofweek|crsdeptime|crsarrtime|crselapsedtime|origin|dest|distance|cancelled|
+----+-----+----------+---------+----------+----------+--------------+------+----+--------+---------+
|   0|    0|         0|        0|         0|         0|             0|     0|   0|       0|        0|
+----+-----+----------+---------+----------+----------+--------------+------+----+--------+---------+



In [20]:
df.show(10)

+----+-----+----------+---------+----------+----------+--------------+------+----+--------+---------+
|year|month|dayofmonth|dayofweek|crsdeptime|crsarrtime|crselapsedtime|origin|dest|distance|cancelled|
+----+-----+----------+---------+----------+----------+--------------+------+----+--------+---------+
|2017|   11|        12|        7|    2055.0|    2205.0|         130.0|   DCA| MKE|   634.0|      0.0|
|2017|   11|        12|        7|    1430.0|    1535.0|         125.0|   DCA| MKE|   634.0|      0.0|
|2017|   11|        12|        7|    1305.0|    1500.0|         175.0|   DCA| MSY|   969.0|      0.0|
|2017|   11|        12|        7|    1855.0|    2050.0|         175.0|   DCA| MSY|   969.0|      0.0|
|2017|   11|        12|        7|    1645.0|    1845.0|         180.0|   DCA| OMA|  1012.0|      0.0|
|2017|   11|        12|        7|    1955.0|    2115.0|          80.0|   DCA| PVD|   356.0|      0.0|
|2017|   11|        12|        7|    1145.0|    1305.0|          80.0|   DCA| PVD|

In [21]:
from pyspark.sql.functions import sum

df.select(sum("cancelled")).show()

+--------------+
|sum(cancelled)|
+--------------+
|      265122.0|
+--------------+



2. Decompose time features

In [22]:
df = df \
    .withColumn("month_sin", F.sin(2*math.pi * F.col("month") / 12.0))\
    .withColumn("month_cos", F.cos(2*math.pi * F.col("month") / 12.0))\
    .withColumn("dayofmonth_sin", F.sin(2*math.pi * F.col("dayofmonth") / 31.0))\
    .withColumn("dayofmonth_cos", F.cos(2*math.pi * F.col("dayofmonth") / 31.0))\
    .withColumn("dayofweek_sin", F.sin(2*math.pi * F.col("dayofweek") / 7.0))\
    .withColumn("dayofweek_cos", F.cos(2*math.pi * F.col("dayofweek") / 7.0))

df.show(10)

+----+-----+----------+---------+----------+----------+--------------+------+----+--------+---------+-------------------+------------------+------------------+-------------------+--------------------+-------------+
|year|month|dayofmonth|dayofweek|crsdeptime|crsarrtime|crselapsedtime|origin|dest|distance|cancelled|          month_sin|         month_cos|    dayofmonth_sin|     dayofmonth_cos|       dayofweek_sin|dayofweek_cos|
+----+-----+----------+---------+----------+----------+--------------+------+----+--------+---------+-------------------+------------------+------------------+-------------------+--------------------+-------------+
|2017|   11|        12|        7|    2055.0|    2205.0|         130.0|   DCA| MKE|   634.0|      0.0|-0.5000000000000004|0.8660254037844384|0.6513724827222223|-0.7587581226927909|-2.44929359829470...|          1.0|
|2017|   11|        12|        7|    1430.0|    1535.0|         125.0|   DCA| MKE|   634.0|      0.0|-0.5000000000000004|0.8660254037844384|

In [23]:
def split_hhmm(colname):
    return (
        F.floor(F.col(colname) / 100).cast(IntegerType()),
        (F.col(colname) % 100).cast(IntegerType())
    )

In [24]:
time_features = ['crsdeptime', 'crsarrtime']

for tf in time_features:
    hr, mn = split_hhmm(tf)
    df = df \
        .withColumn(f"{tf}_hr_sin", F.sin(2*math.pi * hr / 24.0)) \
        .withColumn(f"{tf}_hr_cos", F.cos(2*math.pi * hr / 24.0)) \
        .withColumn(f"{tf}_mn_sin", F.sin(2*math.pi * mn / 60.0)) \
        .withColumn(f"{tf}_mn_cos", F.cos(2*math.pi * mn / 60.0))

df.show(10)

+----+-----+----------+---------+----------+----------+--------------+------+----+--------+---------+-------------------+------------------+------------------+-------------------+--------------------+-------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+--------------------+
|year|month|dayofmonth|dayofweek|crsdeptime|crsarrtime|crselapsedtime|origin|dest|distance|cancelled|          month_sin|         month_cos|    dayofmonth_sin|     dayofmonth_cos|       dayofweek_sin|dayofweek_cos|  crsdeptime_hr_sin|   crsdeptime_hr_cos|   crsdeptime_mn_sin|   crsdeptime_mn_cos|  crsarrtime_hr_sin|   crsarrtime_hr_cos|  crsarrtime_mn_sin|   crsarrtime_mn_cos|
+----+-----+----------+---------+----------+----------+--------------+------+----+--------+---------+-------------------+------------------+------------------+-------------------+--------------------+-------------+-------------------+------

3. Categorical encoding

In [25]:
cats = ["origin", "dest", "year"]
num_feats = ["crselapsedtime", "distance"]

cyc_feats = (
    ["month_sin", "month_cos", "dayofmonth_sin", "dayofmonth_cos", "dayofweek_sin", "dayofweek_cos"] +
    [f"{tf}_{p}_{axis}"
     for tf in time_features
     for p in ["hr", "mn"]
     for axis in ["sin", "cos"]]
)

all_numeric = num_feats + cyc_feats

In [26]:
indexers = [
    StringIndexer(inputCol=c, outputCol=f"{c}_idx", handleInvalid="keep")
    for c in cats
]
ohe = OneHotEncoder(
    inputCols=[f"{c}_idx" for c in cats],
    outputCols=[f"{c}_ohe" for c in cats],
    dropLast=True
)

In [27]:
pre_vector_stages = indexers + [ohe]
pre_vector_pipeline = Pipeline(stages=pre_vector_stages)
pre_vector_model = pre_vector_pipeline.fit(df)

In [28]:
df_raw_plus_ohe = pre_vector_model.transform(df)
intermediate_cols = all_numeric + [f"{c}_ohe" for c in cats] + ["cancelled"]
df_ohe = df_raw_plus_ohe.select(intermediate_cols)

df_ohe.createOrReplaceTempView("flight_features_intermediate")
spark.sql("SELECT * FROM flight_features_intermediate LIMIT 10").show(truncate=False)

25/05/08 16:59:21 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------+--------+-------------------+------------------+------------------+-------------------+-----------------------+-------------+-------------------+-----------------------+--------------------+-----------------------+-------------------+-----------------------+-------------------+-----------------------+----------------+----------------+-------------+---------+
|crselapsedtime|distance|month_sin          |month_cos         |dayofmonth_sin    |dayofmonth_cos     |dayofweek_sin          |dayofweek_cos|crsdeptime_hr_sin  |crsdeptime_hr_cos      |crsdeptime_mn_sin   |crsdeptime_mn_cos      |crsarrtime_hr_sin  |crsarrtime_hr_cos      |crsarrtime_mn_sin  |crsarrtime_mn_cos      |origin_ohe      |dest_ohe        |year_ohe     |cancelled|
+--------------+--------+-------------------+------------------+------------------+-------------------+-----------------------+-------------+-------------------+-----------------------+--------------------+-----------------------+----------------

4. Assemble numeric + cyclical features, then scale

In [29]:
assembler = VectorAssembler(inputCols=all_numeric + [f"{c}_ohe" for c in cats], outputCol="features_raw")
scaler = StandardScaler(inputCol="features_raw", outputCol="features", withMean=True, withStd=True)

In [30]:
pipeline = Pipeline(stages=indexers + [ohe, assembler, scaler])

In [31]:
model = pipeline.fit(df)
df_prepared = model.transform(df).select("features", "cancelled")

In [32]:
df_prepared.show(10)

+--------------------+---------+
|            features|cancelled|
+--------------------+---------+
|[-0.1884130372273...|      0.0|
|[-0.2546376588361...|      0.0|
|[0.40760855725178...|      0.0|
|[0.40760855725178...|      0.0|
|[0.47383317886057...|      0.0|
|[-0.8506592533152...|      0.0|
|[-0.8506592533152...|      0.0|
|[-0.0559637940097...|      0.0|
|[0.01026082759904...|      0.0|
|[-0.0559637940097...|      0.0|
+--------------------+---------+
only showing top 10 rows



5. Balance dataset by down-sampling the majority class

In [33]:
counts = df_prepared.groupBy("cancelled").count().collect()
cnts = {row["cancelled"]: row["count"] for row in counts}
print(cnts)

{0.0: 18240580, 1.0: 265122}


In [34]:
min_class = min(cnts, key=cnts.get)
maj_class = max(cnts, key=cnts.get)
fraction = float(cnts[min_class]) / float(cnts[maj_class])
fractions = {min_class: 1.0, maj_class: fraction}
print(fractions)

{1.0: 1.0, 0.0: 0.014534735189341567}


In [35]:
df_balanced = df_prepared.sampleBy("cancelled", fractions, seed=42)

In [36]:
df_balanced.show(10)

+--------------------+---------+
|            features|cancelled|
+--------------------+---------+
|[0.01026082759904...|      0.0|
|[0.01026082759904...|      0.0|
|[-0.2546376588361...|      0.0|
|[-0.8506592533152...|      0.0|
|[-0.0559637940097...|      0.0|
|[0.27515931403420...|      0.0|
|[-1.0493331181415...|      0.0|
|[-1.1155577397503...|      1.0|
|[1.59965174621000...|      0.0|
|[0.01026082759904...|      0.0|
+--------------------+---------+
only showing top 10 rows



In [37]:
print(f"Number of rows after balancing dataset: {df_balanced.count()}")

Number of rows after balancing dataset: 531167


In [38]:
counts = df_balanced.groupBy("cancelled").count().collect()
cnts = {row["cancelled"]: row["count"] for row in counts}
print(cnts)

{0.0: 266045, 1.0: 265122}


6. Split dataset into train / val (with stratify)

In [39]:
df_class_0 = df_balanced.filter(F.col("cancelled") == 0.0)
df_class_1 = df_balanced.filter(F.col("cancelled") == 1.0)
train_0, val_0 = df_class_0.randomSplit([0.8, 0.2], seed=42)
train_1, val_1 = df_class_1.randomSplit([0.8, 0.2], seed=42)

train_df = train_0.unionAll(train_1)
val_df = val_0.unionAll(val_1)
print("Train size:", train_df.count(), "Valid size:", val_df.count())

Train size: 425472 Valid size: 105695


In [40]:
counts = train_df.groupBy("cancelled").count().collect()
cnts = {row["cancelled"]: row["count"] for row in counts}
print(cnts)

{0.0: 213099, 1.0: 212373}


In [41]:
train_df.show(10)

+--------------------+---------+
|            features|cancelled|
+--------------------+---------+
|[-1.6321097882989...|      0.0|
|[-1.6188648639771...|      0.0|
|[-1.5393953180466...|      0.0|
|[-1.5393953180466...|      0.0|
|[-1.4996605450813...|      0.0|
|[-1.4864156207596...|      0.0|
|[-1.4864156207596...|      0.0|
|[-1.4864156207596...|      0.0|
|[-1.4731706964378...|      0.0|
|[-1.4731706964378...|      0.0|
+--------------------+---------+
only showing top 10 rows



In [42]:
val_df.show(10)

+--------------------+---------+
|            features|cancelled|
+--------------------+---------+
|[-1.6056199396554...|      0.0|
|[-1.4864156207596...|      0.0|
|[-1.4864156207596...|      0.0|
|[-1.4731706964378...|      0.0|
|[-1.4599257721160...|      0.0|
|[-1.4599257721160...|      0.0|
|[-1.4599257721160...|      0.0|
|[-1.4466808477943...|      0.0|
|[-1.4466808477943...|      0.0|
|[-1.4466808477943...|      0.0|
+--------------------+---------+
only showing top 10 rows



7. Save the data

In [43]:
%cd ..
%ls

/home/team15/BigData-project
data/    models/     output/    recovered-file-0.csv  scripts/  sql/
main.sh  notebooks/  README.MD  requirements.txt      secrets/  venv/


In [45]:
import os

os.makedirs("data", exist_ok=True)

In [87]:
def run(command):
    return os.popen(command).read()

In [47]:
train_df.select("features", "cancelled")\
    .coalesce(1)\
    .write\
    .mode("overwrite")\
    .format("json")\
    .save("project/data/train")

run("hdfs dfs -cat project/data/train/*.json > data/train.json")

''

In [48]:
val_df.select("features", "cancelled")\
    .coalesce(1)\
    .write\
    .mode("overwrite")\
    .format("json")\
    .save("project/data/val")

run("hdfs dfs -cat project/data/val/*.json > data/val.json")

''

### ML modeling

#### Model 1: Logistic regression

In [49]:
label_col = "cancelled"
features_col = "features"

lr = LogisticRegression(labelCol=label_col, featuresCol=features_col)

In [50]:
paramGrid_lr = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 0.5]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .addGrid(lr.maxIter, [10, 50]) \
    .build()

In [51]:
evaluator1 = BinaryClassificationEvaluator(labelCol=label_col, rawPredictionCol="rawPrediction", metricName="areaUnderROC")

In [52]:
cv_lr = CrossValidator(estimator=lr,
                       estimatorParamMaps=paramGrid_lr,
                       evaluator=evaluator1,
                       numFolds=3,
                       parallelism=5,
                       seed=42)

In [53]:
print("Training Logistic Regression with Cross-Validation...")
cv_model_lr = cv_lr.fit(train_df)
print("Logistic Regression training complete.")

Training Logistic Regression with Cross-Validation...


25/05/08 17:43:50 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/05/08 18:31:53 ERROR TransportClient: Failed to send RPC RPC 6296566509472130664 to /10.100.30.60:57858: io.netty.channel.StacklessClosedChannelException
io.netty.channel.StacklessClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(Object, ChannelPromise)(Unknown Source)
25/05/08 18:31:53 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_5139_1 !
25/05/08 18:31:53 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_5424_7 !
25/05/08 18:31:53 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_5424_2 !
25/05/08 18:31:53 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_5175_6 !
25/05/08 18:31:53 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_5139_12 !
25/05/08 18:31:53 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_5424_5 !
25/05/08 18:31:53 

Logistic Regression training complete.


In [54]:
best_lr_model = cv_model_lr.bestModel

In [55]:
print("\nBest Logistic Regression Model Parameters:")
for param in best_lr_model.extractParamMap():
    if param.name in ['regParam', 'elasticNetParam', 'maxIter']:
        print(f"  {param.name}: {best_lr_model.getOrDefault(param)}")


Best Logistic Regression Model Parameters:
  elasticNetParam: 0.0
  maxIter: 50
  regParam: 0.01


In [56]:
predictions_lr = best_lr_model.transform(val_df)

In [57]:
auc_lr_test = evaluator1.evaluate(predictions_lr)
print(f"Logistic Regression - Test AUC: {auc_lr_test}")

Logistic Regression - Test AUC: 0.6958213003410854


In [58]:
def get_detailed_metrics(predictions_df, model_name="Model"):
    print(f"\nDetailed Metrics for {model_name} on Test Data:")

    preds_and_labels = predictions_df.select(
        F.col("prediction").cast(DoubleType()),
        F.col(label_col).cast(DoubleType())
    ).rdd.map(lambda r: (r[0], r[1]))

    metrics = MulticlassMetrics(preds_and_labels)
    print(f"  Confusion Matrix:\n{metrics.confusionMatrix().toArray()}")
    print(f"  Precision (Cancelled=1.0): {metrics.precision(1.0):.4f}")
    print(f"  Recall (Cancelled=1.0): {metrics.recall(1.0):.4f}")
    print(f"  F1-Score (Cancelled=1.0): {metrics.fMeasure(1.0):.4f}")
    print(f"  Accuracy: {metrics.accuracy:.4f}")
    return metrics.accuracy, metrics.precision(1.0), metrics.recall(1.0), metrics.fMeasure(1.0)


lr_accuracy, lr_precision, lr_recall, lr_f1 = get_detailed_metrics(predictions_lr, "Logistic Regression")


Detailed Metrics for Logistic Regression on Test Data:


/usr/local/lib/python3.11/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


  Confusion Matrix:
[[33428. 19518.]
 [18366. 34383.]]
  Precision (Cancelled=1.0): 0.6379
  Recall (Cancelled=1.0): 0.6518
  F1-Score (Cancelled=1.0): 0.6448
  Accuracy: 0.6416


 Save models

In [59]:
from pprint import pprint

model_output_path_lr = f"project/models/logistic_regression_model"
best_lr_model = cv_model_lr.bestModel
pprint(best_lr_model.extractParamMap())

{Param(parent='LogisticRegression_9942d619e73a', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LogisticRegression_9942d619e73a', name='fitIntercept', doc='whether to fit an intercept term.'): True,
 Param(parent='LogisticRegression_9942d619e73a', name='labelCol', doc='label column name.'): 'cancelled',
 Param(parent='LogisticRegression_9942d619e73a', name='standardization', doc='whether to standardize the training features before fitting the model.'): True,
 Param(parent='LogisticRegression_9942d619e73a', name='predictionCol', doc='prediction column name.'): 'prediction',
 Param(parent='LogisticRegression_9942d619e73a', name='maxBlockSizeInMB', doc='maximum memory in MB for stacking input data into blocks. Data is stacked within partitions. If more than remaining data size in a partition then it is adjusted to the data size. Default 0.0 represents cho

In [60]:
best_lr_model.write().overwrite().save(model_output_path_lr)
run("hdfs dfs -get project/models/logistic_regression_model models/logistic_regression_model")
print(f"Saved best Logistic Regression model to: {model_output_path_lr}")

Saved best Logistic Regression model to: project/models/logistic_regression_model


Save predictions

In [61]:
predictions_output_path_lr = f"project/output/lr_model_predictions.csv"

predictions_lr.select(label_col, "prediction") \
    .coalesce(1) \
    .write.mode("overwrite").format("csv") \
    .option("sep", ",")\
    .option("header", "true") \
    .save(predictions_output_path_lr)

run("hdfs dfs -cat project/output/lr_model_predictions.csv/*.csv > output/lr_model_predictions.csv")
print(f"Saved Logistic Regression predictions to: {predictions_output_path_lr}")

Saved Logistic Regression predictions to: project/output/lr_model_predictions.csv


#### Model 2: Random Forest

In [71]:
rf = RandomForestClassifier(labelCol=label_col, featuresCol=features_col, seed=42)

In [72]:
paramGrid_rf = ParamGridBuilder() \
    .addGrid(rf.numTrees, [20, 50]) \
    .addGrid(rf.maxDepth, [5, 10, 15]) \
    .addGrid(rf.featureSubsetStrategy, ["sqrt", "log2"]) \
    .build()

In [73]:
evaluator2 = BinaryClassificationEvaluator(labelCol=label_col, rawPredictionCol="rawPrediction", metricName="areaUnderROC")

In [74]:
cv_rf = CrossValidator(estimator=rf,
                       estimatorParamMaps=paramGrid_rf,
                       evaluator=evaluator2,
                       numFolds=3,
                       parallelism=5,
                       seed=42)

In [75]:
print("Training Random Forest with Cross-Validation...")
cv_model_rf = cv_rf.fit(train_df)
print("Random Forest training complete.")

Training Random Forest with Cross-Validation...


25/05/08 20:18:51 WARN CacheManager: Asked to cache already cached data.
25/05/08 20:18:51 WARN CacheManager: Asked to cache already cached data.
25/05/08 20:31:21 WARN DAGScheduler: Broadcasting large task binary with size 1247.4 KiB
25/05/08 20:31:24 WARN DAGScheduler: Broadcasting large task binary with size 1247.4 KiB
25/05/08 20:31:27 WARN DAGScheduler: Broadcasting large task binary with size 1247.4 KiB
25/05/08 20:31:31 WARN DAGScheduler: Broadcasting large task binary with size 1247.4 KiB
25/05/08 20:31:42 WARN DAGScheduler: Broadcasting large task binary with size 1579.1 KiB
25/05/08 20:31:45 WARN DAGScheduler: Broadcasting large task binary with size 1579.2 KiB
25/05/08 20:31:57 WARN DAGScheduler: Broadcasting large task binary with size 2001.2 KiB
25/05/08 20:32:01 WARN DAGScheduler: Broadcasting large task binary with size 2001.2 KiB
25/05/08 20:32:10 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
25/05/08 20:32:13 WARN DAGScheduler: Broadcasting large 

Random Forest training complete.


In [76]:
best_rf_model = cv_model_rf.bestModel

In [77]:
print("\nBest Random Forest Model Parameters:")
for param in best_rf_model.extractParamMap():
     if param.name in ['numTrees', 'maxDepth', 'featureSubsetStrategy']:
        print(f"  {param.name}: {best_rf_model.getOrDefault(param)}")


Best Random Forest Model Parameters:
  featureSubsetStrategy: sqrt
  maxDepth: 15
  numTrees: 50


In [78]:
predictions_rf = best_rf_model.transform(val_df)

In [79]:
auc_rf_test = evaluator2.evaluate(predictions_rf)
print(f"Random Forest - Test AUC: {auc_rf_test}")

25/05/08 21:52:03 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB


Random Forest - Test AUC: 0.7649094722448742


In [80]:
rf_accuracy, rf_precision, rf_recall, rf_f1 = get_detailed_metrics(predictions_rf, "Random Forest")


Detailed Metrics for Random Forest on Test Data:


/usr/local/lib/python3.11/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
25/05/08 21:55:07 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB
25/05/08 21:55:40 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB


  Confusion Matrix:
[[36710. 16236.]
 [15939. 36810.]]
  Precision (Cancelled=1.0): 0.6939
  Recall (Cancelled=1.0): 0.6978
  F1-Score (Cancelled=1.0): 0.6959
  Accuracy: 0.6956


Save model

In [81]:
model_output_path_rf = f"project/models/random_forest_model"
best_rf_model = cv_model_rf.bestModel
pprint(best_rf_model.extractParamMap())

{Param(parent='RandomForestClassifier_7dbb85a09305', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False,
 Param(parent='RandomForestClassifier_7dbb85a09305', name='bootstrap', doc='Whether bootstrap samples are used when building trees.'): True,
 Param(parent='RandomForestClassifier_7dbb85a09305', name='numTrees', doc='Number of trees to train (>= 1).'): 50,
 Param(parent='RandomForestClassifier_7dbb85a09305', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.'): 1,
 Param(parent='RandomForestClassifier_7dbb85a09305', name='seed', do

In [82]:
best_rf_model.write().overwrite().save(model_output_path_rf)
run("hdfs dfs -get project/models/random_forest_model models/random_forest_model")
print(f"Saved best Random Forest model to: {model_output_path_rf}")

25/05/08 21:59:06 WARN TaskSetManager: Stage 5123 contains a task of very large size (2231 KiB). The maximum recommended task size is 1000 KiB.


Saved best Random Forest model to: project/models/random_forest_model


Save predictions

In [83]:
predictions_output_path_rf = f"project/output/rf_model_predictions.csv"

predictions_rf.select(label_col, "prediction") \
    .coalesce(1) \
    .write.mode("overwrite").format("csv") \
    .option("sep", ",")\
    .option("header", "true") \
    .save(predictions_output_path_rf)

run("hdfs dfs -cat project/output/rf_model_predictions.csv/*.csv > output/rf_model_predictions.csv")
print(f"Saved Random Forest predictions to: {predictions_output_path_rf}")

25/05/08 21:59:09 WARN DAGScheduler: Broadcasting large task binary with size 4.7 MiB


Saved Random Forest predictions to: project/output/rf_model_predictions.csv


#### Comapare best models

In [ ]:
models = [
    [str(best_lr_model), auc_lr_test, lr_accuracy, lr_precision, lr_recall, lr_f1],
    [str(best_rf_model), auc_rf_test, rf_accuracy, rf_precision, rf_recall, rf_f1]
]

df = spark.createDataFrame(models, ["Model", "AUC", "Accuracy", "Precision", "Recall", "F1-score"])
df.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------+------------------+------------------+------------------+------------------+------------------+
|Model                                                                                                               |AUC               |Accuracy          |Precision         |Recall            |F1-score          |
+--------------------------------------------------------------------------------------------------------------------+------------------+------------------+------------------+------------------+------------------+
|LogisticRegressionModel: uid=LogisticRegression_9942d619e73a, numClasses=2, numFeatures=741                         |0.6958213003410854|0.6415724490278631|0.6378916903211443|0.6518227833703009|0.6447819971870605|
|RandomForestClassificationModel: uid=RandomForestClassifier_7dbb85a09305, numTrees=50, numClasses=2, numFeatures=741|0.7649094722448742|0.69558

In [85]:
df.coalesce(1)\
    .write\
    .mode("overwrite")\
    .format("csv")\
    .option("sep", ",")\
    .option("header","true")\
    .save("project/output/evaluation.csv")

In [86]:
run("hdfs dfs -cat project/output/evaluation.csv/*.csv > output/evaluation.csv")

''